# Naive First Approach
This notebook focus on create the first units in the game and deliver a first agent more developed than a "simple agent". 
### 1 - Add the data to /kaggle/input folder
a) Click "+ Add data"
b) Select "Competition Data"
c) Find "Lux AI"
d) Click "Add" button.

Input folder will show "lux-ai-2021" folder with "lux" subfolder.

In order to generate the file to submission you must: (1)  comment the simulation cell; (2) uncomment the first line of agent cell and (3) uncomment the cell that generates the submisison file.

To simulate it locally do the opposite of above.

Version 1 is ready to run loccaly. Version 2 ready to submit.

### 2- Then, copy the folder to /kaggle/output folder to be used in submission file.

In [ ]:
!cp -r ../input/lux-ai-2021/* .

### 3 - Run the agent below and check the results.

In the cell below, about the first line '%%writefile agent.py'

a) Comment the first line to run agent locally. 

b) Uncomment the first line to generate de file to submit

In [ ]:
%%writefile agent.py


from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

### Define helper functions

# +++++++++++++++++++++++++++++++++++++++++++++++++
#   FIND RESOURCES IN THE MAP
# +++++++++++++++++++++++++++++++++++++++++++++++++
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles


# +++++++++++++++++++++++++++++++++++++++++++++++++
#   FIND RESOURCES CLOSEST TO UNIT FULL OF CARGO
# +++++++++++++++++++++++++++++++++++++++++++++++++
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    for resource_tile in resource_tiles:
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            if (resource_tile.resource.type=='uranium') & (player.researched_uranium()): 
                closest_dist = dist
                closest_resource_tile = resource_tile
            elif (resource_tile.resource.type=='coal') & (player.researched_coal()): 
                closest_dist = dist
                closest_resource_tile = resource_tile
            elif (resource_tile.resource.type=='wood'):
                closest_dist = dist
                closest_resource_tile = resource_tile
    return closest_resource_tile

# +++++++++++++++++++++++++++++++++++++++++++++++++
#   FIND CITYTILE TO DELIVERY RESOURCES
# +++++++++++++++++++++++++++++++++++++++++++++++++
def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

# ===========================================
#  FIND THE TILES ADJACENT TO GIVEN POSITION
# ============================================
def adjacent_tiles(pos):
    ''' Get adjacent tiles given a tile'''
    x1,y1 = pos.x, pos.y
    w, h = game_state.map.width, game_state.map.height
    adjacent_tiles_pos = [(x1-1,y1),(x1+1,y1),(x1,y1+1),(x1,y1-1)]
    return [ game_state.map.get_cell(x,y) for (x,y) in adjacent_tiles_pos if (x<w)&(y<w)&(x>=0)&(y>=0) ]

# ==============================================================
#  FIND THE TILE TO BUILD ADJACENT TO CLOSEST CITYTILE
# ==============================================================
def building_tiles(player, pos):
    tile_to_build = None
    cell_position = game_state.map.get_cell_by_pos(pos)
    if (cell_position.has_resource() == False):
        tile_to_build = game_state.map.get_cell_by_pos(pos)
    else:
        tiles_next_pos = adjacent_tiles(pos)
        for cell in tiles_next_pos:
            cell_found = False
            if (cell.has_resource()==False) & (cell.citytile==None) & (cell_found==False):
                tile_to_build = cell
                cell_found = True
    return tile_to_build
 
# ======================================================
#         AGENT
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++
game_state = None

def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])

    ### AI Code goes down here! ### 
    # =======================================================================
    #          INITIAL CONDITIONS
    # ======================================================================
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    actions = []
    resource_tiles = find_resources(game_state)

    # =======================================================================
    #          INITIAL ORDERS
    # ======================================================================
    tiles_in_cities = [ ct for k,city in player.cities.items() for ct in city.citytiles ]
    for k, city in player.cities.items():
#         if (len(tiles_in_cities)>(len(player.units))):
        for i,city_tile in enumerate(city.citytiles):
            if city_tile.can_act():
                if (i%3==0):
                    action = city_tile.build_worker()
                    print("<<< Train >>>", action)
                    actions.append(action)
                elif (i%3==1):
                    action = city_tile.research()
                    print('<<< Research >>> ')
                    actions.append(action)
                else:
                    action = city_tile.build_cart()
                    print("<<< Train Cart >>>", action)
                    actions.append(action)
                        
#     # Uncomment line below to see the  the turn and situation of city  
#     print("\nTURN: ", game_state.turn )    
#     print("Units:",  len (player.units ),"|| City :" , len(player.cities.items())"|| Research points: ", player.research_points)

    # =======================================================================
    #    Loop through units and give orders (workers and carts) 
    # =======================================================================
    for i, unit in enumerate(player.units):
        if unit.is_worker() and unit.can_act():
            if unit.get_cargo_space_left()>0:
                closest_resource_tile = find_closest_resources(unit.pos,player,resource_tiles)
                next_direction = unit.pos.direction_to(closest_resource_tile.pos)
                action = unit.move(next_direction)
                actions.append(action)
            else:  
                closest_city_tile = find_closest_city_tile(unit.pos, player)
                required_fuel_city =player.cities[closest_city_tile.cityid].get_light_upkeep()*10
                if player.cities[closest_city_tile.cityid].fuel < required_fuel_city:
                    next_direction = unit.pos.direction_to(closest_city_tile.pos)
                    action = unit.move(next_direction)
                    actions.append(action)
                else:  
                    tile_to_build = building_tiles(player, unit.pos)
                    if tile_to_build != None:
                        if unit.pos.equals(tile_to_build.pos):
                            if unit.can_build(game_state.map):
                                action = unit.build_city()
                                actions.append(action)
                        else:
                            next_direction = unit.pos.direction_to(tile_to_build.pos)
                            action = unit.move(next_direction)
                            actions.append(action) 
                            
                            
        if unit.is_cart() and unit.can_act():   
            middle_cell = game_state.map.get_cell(width//2, height//2)
            action = unit.move(unit.pos.direction_to(middle_cell.pos) )
            actions.append(action)
#     # Uncomment line below to see actions given by agent    
#     print('Actions: ', actions)         
    return actions

### 4 - Run the cell below to simulate the agent with simple agent

In the cell below, comment all lines below to generate file to submit.

In [ ]:
# from kaggle_environments import make
# env = make("lux_ai_2021", configuration={"seed": 1000, "loglevel": 2, "annotations": True}, debug=True)
# # steps = env.run([agent, agent])
# steps = env.run([agent, "simple_agent"])
# env.render(mode="ipython", width = 1000, height = 800)

### 5 - Generate the file to sumbission

Uncomment the cell below to generate file to sumbission; "submission.tar.gz" 

In [ ]:
# Submission file
!tar -czvf submission.tar.gz __init__.py agent.py main.py lux

This notebook aims to help you begin in the challenge such as:

- print annotation when run locally;
- build worker, cart and citytiles;
- set actions to units (workers and carts);
- how to research, build workers and carts;
- check warnings given programm.

It returns error in submission in some maps.

### I hope it helps to start.